In [ ]:
!pip install keras==2.3.1

In [3]:
!pip install tensorflow==2.2.0

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install segmentation-models

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install opencv-python

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [6]:
import segmentation_models as sm

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [15]:

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv
from tqdm import tqdm
import SaveLoadModel
import BatchManager

# def split_image(orig, mask, IMG_SIZE):
#     assert orig.shape[0] == mask.shape[0]
#     assert orig.shape[1] == mask.shape[1]
#     data = []
#     label = []
#     y = 0
#     x = 0
#     for y in range(0, orig.shape[0] - IMG_SIZE, IMG_SIZE):
#         for x in range(0, orig.shape[1] - IMG_SIZE, IMG_SIZE):
#             current_data = np.empty((IMG_SIZE, IMG_SIZE,3))
#             current_data[:,:] = orig[y:y + IMG_SIZE, x:x + IMG_SIZE]
#             current_label = np.empty((IMG_SIZE, IMG_SIZE, 3))
#             for j in range(IMG_SIZE):
#                 for i in range(IMG_SIZE):
#                     c = mask[y + j, x + i]
#                     if (c <= 100).all():
#                         current_label[j, i] = np.array([1, 0, 0])
#                     elif (c >= 150).all():
#                         current_label[j, i] = np.array([0, 1, 0])
#                     else:
#                         current_label[j, i] = np.array([0, 0, 1])
#             data.append(current_data)
#             label.append(current_label)
#     return data, label
#
#
# def my_load_data():
#     DATADIR = "C:/Users/Sasha/Documents/Bubbles/bubbles and masks"
#     img_postfix = "original.jpg"
#     mask_postfix = "mask.jpg"
#     IMG_SIZE = 224
#     training_data = []
#     training_mask = []
#     val_data = []
#     val_mask = []
#
#     path = DATADIR
#
#
#     for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
#         if img.endswith(img_postfix):
#             mask = img[:-len(img_postfix)] + mask_postfix
#             try:
#                 img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_COLOR)
#                 mask_array = cv2.imread(os.path.join(path,mask) ,cv2.IMREAD_COLOR)
#                 # img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#                 # plt.imshow(img_array)  # graph it
#                 # plt.show()  # display!
#                 data, label = split_image(img_array, mask_array, IMG_SIZE)
#                 training_data +=data
#                 training_mask += label
#             except OSError as e:
#                print("OSErrroBad img most likely", e, os.path.join(path,img))
#             except Exception as e:
#                print("general exception", e, os.path.join(path,img))
#     val_data = training_data
#     val_mask = training_mask
#     return np.stack(training_data), np.stack(training_mask), np.stack(val_data), np.stack(val_mask)

def main():
    BACKBONE = 'resnet50'
    preprocess_input = sm.get_preprocessing(BACKBONE)

    DATADIR = "F:/Downloads/Plugs_data/1/data"
    x_train, y_train, _ = BatchManager.generate_xy_folder(DATADIR, IMG_SIZE=32 * 8, BORDER=32 * 1)
    VALDIR = "F:/Downloads/Plugs_data/1/masks"
    x_val, y_val, _ = BatchManager.generate_xy_folder(VALDIR, IMG_SIZE=32 * 8, BORDER=32 * 1)

    # preprocess input
    x_train = preprocess_input(x_train)
    x_val = preprocess_input(x_val)

    # define model
    model = sm.Unet(BACKBONE, classes=3, activation='softmax', encoder_weights='imagenet')
    model.compile(
        'Adam',
        loss=sm.losses.bce_jaccard_loss,
        metrics=[sm.metrics.iou_score],
    )

    # fit model
    # if you use data generator use model.fit_generator(...) instead of model.fit(...)
    # more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator
    model.fit(
       x=x_train,
       y=y_train,
       batch_size=16,
       epochs=20,
       validation_data=(x_val, y_val),
    )

    SaveLoadModel.SaveModel(model, "C:/Users/Sasha/Documents/Bubbles/model", "5")

if __name__ == "__main__":
    main()
    
print(5)

100%|██████████| 761/761 [00:00<00:00, 760148.93it/s]


ValueError: need at least one array to stack